# AllenNLP 
## Must implement the following classes:
1. DatasetReader
2. Model

## Step 1.  DatasetReader

* Must implement __init__
  * must pass TokenIndexer(s) as argument to initialize
* Must implemment _read method
  * read files, http_pages etc
  * pre-process the text 
* Must implement text_to_instance 
  * convert text into instances =[ Token, Label ] 


In [32]:
from typing import Iterator, List, Dict

import torch
import torch.optim as optim
import numpy as np

from allennlp.data import Instance
from allennlp.data import Token
from allennlp.data import DatasetReader

from allennlp.data.fields import TextField, SequenceLabelField

from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer

from allennlp.common.file_utils import cached_path


class PosDatasetReader(DatasetReader):
   
    def __init__(self,token_indexers:Dict[str,TokenIndexer]=None)->None:

        
        super().__init__(lazy=False)
       
        self.token_indexers = token_indexers or {"tokens":SingleIdTokenIndexer()}
        
    
    def _read(self,file_path:str) -> Iterator[Instance]:

        with open(file_path) as f :

            for line in f:
            
                pairs = line.strip().split()
                
                sentence, tags = zip(*(pair.split("###")for pair in pairs))
                
                print("sentence : " , sentence , " tags : " , tags)
                
                yield self.text_to_instance([Token(word) for word in sentence],tags)
                
    
    def text_to_instance(self,tokens: List[Token], tags: List[str]=None)->Instance:
        
        fields = dict()
        
        sentence_field = TextField(tokens,self.token_indexers)
        
        fields["sentence"] = sentence_field # {"sentence":sentence_field}
 
        # Make label fields if tag is given
        if tags:
            label_field = SequenceLabelField(labels = tags, sequence_field=sentence_field)
            fields["labels"] = label_field
                
        # return instance list (= dataset)
        return Instance(fields)
    

    

In [33]:
# Read dataset 

# 1. Instantiate DatasetReader
reader = PosDatasetReader()

# 2. Call read to make dataset

train_dataset = reader.read(cached_path(
    'https://raw.githubusercontent.com/allenai/allennlp'
    '/master/tutorials/tagger/training.txt'))

validation_dataset = reader.read(cached_path(
    'https://raw.githubusercontent.com/allenai/allennlp'
    '/master/tutorials/tagger/validation.txt'))

2it [00:00, 1060.77it/s]


sentence :  ('The', 'dog', 'ate', 'the', 'apple')  tags :  ('DET', 'NN', 'V', 'DET', 'NN')
sentence :  ('Everybody', 'read', 'that', 'book')  tags :  ('NN', 'V', 'DET', 'NN')


2it [00:00, 1277.00it/s]

sentence :  ('The', 'dog', 'read', 'the', 'apple')  tags :  ('DET', 'NN', 'V', 'DET', 'NN')
sentence :  ('Everybody', 'ate', 'that', 'book')  tags :  ('NN', 'V', 'DET', 'NN')


## Step 2. Implement the Model

* Sub-class of torch.nn.Module
* Needs a forward method

  * Must return a dict of tensor outputs
  * Must include dict[loss] for training

* In our model, we'll use : 
  * embedding layer
  * sequence embedder (lstm)
  * feedforward net
  

In [45]:
from allennlp.models import Model

from allennlp.data import Vocabulary

from allennlp.modules import TextFieldEmbedder, Seq2SeqEncoder

from allennlp.training.metrics import CategoricalAccuracy

from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits

class LstmTagger(Model):
    
    
    def __init__(self,
                 word_embeddings : TextFieldEmbedder,
                 encoder: Seq2SeqEncoder,
                 vocab: Vocabulary) -> None:
        
        super().__init__(vocab)
        
        self.word_embeddings = word_embeddings
        
        self.encoder = encoder
        
        self.hidden2tag = torch.nn.Linear(in_features  = encoder.get_output_dim(),
                                          out_features = vocab.get_vocab_size('labels'))
        
        self.accuracy = CategoricalAccuracy()
        
    
    def forward(self,
                sentence: Dict[str,torch.Tensor],
                labels: torch.Tensor = None) -> Dict[str,torch.Tensor]:
        
        mask = get_text_field_mask(sentence)
        
        embeddings = self.word_embeddings(sentence)
        
        encoder_out = self.encoder(embeddings,mask)
        
        tag_logits = self.hidden2tag(encoder_out)
        
        output = {"tag_logits":tag_logits}
        
        if labels is not None:
            self.accuracy(tag_logits, labels, mask)
            output['loss']=sequence_cross_entropy_with_logits(tag_logits,labels,mask)
            
        return output
    
    
    def get_metric(self,
                  reset: bool = False) -> Dict[str,float]:
        
        return {"accuracy": self.accuracy.get_metric(reset)}

# Step 3. Instantiate the model

 * must pass to the model 
   * Vocabulary of dataset
     * to get the size of label vocabulary (== output feature dimension)    
   * type of Embedder to use
   * type of Encoder to use


In [56]:
from allennlp.data import Vocabulary
# 1. Define vocabulary
vocab = Vocabulary.from_instances(train_dataset)
vocabulary_size = vocab.get_vocab_size('tokens')

100%|██████████| 2/2 [00:00<00:00, 1746.90it/s]


In [57]:
# 2. Define embedders
from allennlp.modules import Embedding

from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder

vocabulary_size = input_vocab.get_vocab_size('tokens')

token_embedding = Embedding(num_embeddings=vocabulary_size,
                            embedding_dim = 6)

word_embeddings = BasicTextFieldEmbedder({'tokens':token_embedding})

In [64]:
# 3. Define encoder 

from allennlp.modules.seq2seq_encoders import PytorchSeq2SeqWrapper

module = torch.nn.LSTM(6,6,batch_first=True)

lstm_encoder = PytorchSeq2SeqWrapper(module)

In [65]:
# finally instantiate the model

model = LstmTagger(word_embeddings,lstm_encoder ,vocab)

## Step 4. Train the Model

In [66]:
from allennlp.training import Trainer
from allennlp.data.iterators import BucketIterator



In [67]:
optimizer = optim.SGD(model.parameters(), lr=0.1)

iterator = BucketIterator(batch_size=2,sorting_keys=[("sentence", "num_tokens")])

iterator.index_with(vocab)

trainer = Trainer(model = model,
                  optimizer = optimizer,
                  iterator = iterator,
                  train_dataset = train_dataset,
                  validation_dataset = validation_dataset,
                  patience = 10,
                  num_epochs = 1000)

trainer.train()

loss: 1.1795 ||: 100%|██████████| 1/1 [00:00<00:00, 30.89it/s]
  0%|          | 0/1 [00:00<?, ?it/s]shuffle parameter is set to False, while bucket iterators by definition change the order of your data.
loss: 1.1694 ||: 100%|██████████| 1/1 [00:00<00:00, 187.73it/s]
loss: 1.1693 ||: 100%|██████████| 1/1 [00:00<00:00, 98.74it/s]
  0%|          | 0/1 [00:00<?, ?it/s]shuffle parameter is set to False, while bucket iterators by definition change the order of your data.
loss: 1.1601 ||: 100%|██████████| 1/1 [00:00<00:00, 208.11it/s]
loss: 1.1600 ||: 100%|██████████| 1/1 [00:00<00:00, 114.37it/s]
  0%|          | 0/1 [00:00<?, ?it/s]shuffle parameter is set to False, while bucket iterators by definition change the order of your data.
loss: 1.1516 ||: 100%|██████████| 1/1 [00:00<00:00, 205.03it/s]
loss: 1.1515 ||: 100%|██████████| 1/1 [00:00<00:00, 140.12it/s]
  0%|          | 0/1 [00:00<?, ?it/s]shuffle parameter is set to False, while bucket iterators by definition change the order of your 

loss: 1.0622 ||: 100%|██████████| 1/1 [00:00<00:00, 276.23it/s]
loss: 1.0621 ||: 100%|██████████| 1/1 [00:00<00:00, 131.81it/s]
  0%|          | 0/1 [00:00<?, ?it/s]shuffle parameter is set to False, while bucket iterators by definition change the order of your data.
loss: 1.0613 ||: 100%|██████████| 1/1 [00:00<00:00, 232.00it/s]
loss: 1.0612 ||: 100%|██████████| 1/1 [00:00<00:00, 114.20it/s]
  0%|          | 0/1 [00:00<?, ?it/s]shuffle parameter is set to False, while bucket iterators by definition change the order of your data.
loss: 1.0604 ||: 100%|██████████| 1/1 [00:00<00:00, 179.94it/s]
loss: 1.0603 ||: 100%|██████████| 1/1 [00:00<00:00, 107.54it/s]
  0%|          | 0/1 [00:00<?, ?it/s]shuffle parameter is set to False, while bucket iterators by definition change the order of your data.
loss: 1.0595 ||: 100%|██████████| 1/1 [00:00<00:00, 245.32it/s]
loss: 1.0594 ||: 100%|██████████| 1/1 [00:00<00:00, 113.29it/s]
  0%|          | 0/1 [00:00<?, ?it/s]shuffle parameter is set to Fal

  0%|          | 0/1 [00:00<?, ?it/s]shuffle parameter is set to False, while bucket iterators by definition change the order of your data.
loss: 1.0457 ||: 100%|██████████| 1/1 [00:00<00:00, 250.30it/s]
loss: 1.0455 ||: 100%|██████████| 1/1 [00:00<00:00, 115.27it/s]
  0%|          | 0/1 [00:00<?, ?it/s]shuffle parameter is set to False, while bucket iterators by definition change the order of your data.
loss: 1.0453 ||: 100%|██████████| 1/1 [00:00<00:00, 164.85it/s]
loss: 1.0452 ||: 100%|██████████| 1/1 [00:00<00:00, 131.80it/s]
  0%|          | 0/1 [00:00<?, ?it/s]shuffle parameter is set to False, while bucket iterators by definition change the order of your data.
loss: 1.0449 ||: 100%|██████████| 1/1 [00:00<00:00, 142.62it/s]
loss: 1.0448 ||: 100%|██████████| 1/1 [00:00<00:00, 128.64it/s]
  0%|          | 0/1 [00:00<?, ?it/s]shuffle parameter is set to False, while bucket iterators by definition change the order of your data.
loss: 1.0446 ||: 100%|██████████| 1/1 [00:00<00:00, 164.

loss: 1.0344 ||: 100%|██████████| 1/1 [00:00<00:00, 237.11it/s]
loss: 1.0341 ||: 100%|██████████| 1/1 [00:00<00:00, 94.48it/s]
  0%|          | 0/1 [00:00<?, ?it/s]shuffle parameter is set to False, while bucket iterators by definition change the order of your data.
loss: 1.0339 ||: 100%|██████████| 1/1 [00:00<00:00, 230.03it/s]
loss: 1.0337 ||: 100%|██████████| 1/1 [00:00<00:00, 114.62it/s]
  0%|          | 0/1 [00:00<?, ?it/s]shuffle parameter is set to False, while bucket iterators by definition change the order of your data.
loss: 1.0335 ||: 100%|██████████| 1/1 [00:00<00:00, 218.90it/s]
loss: 1.0333 ||: 100%|██████████| 1/1 [00:00<00:00, 109.57it/s]
  0%|          | 0/1 [00:00<?, ?it/s]shuffle parameter is set to False, while bucket iterators by definition change the order of your data.
loss: 1.0331 ||: 100%|██████████| 1/1 [00:00<00:00, 219.02it/s]
loss: 1.0328 ||: 100%|██████████| 1/1 [00:00<00:00, 133.06it/s]
  0%|          | 0/1 [00:00<?, ?it/s]shuffle parameter is set to Fals

  0%|          | 0/1 [00:00<?, ?it/s]shuffle parameter is set to False, while bucket iterators by definition change the order of your data.
loss: 1.0178 ||: 100%|██████████| 1/1 [00:00<00:00, 174.89it/s]
loss: 1.0174 ||: 100%|██████████| 1/1 [00:00<00:00, 111.86it/s]
  0%|          | 0/1 [00:00<?, ?it/s]shuffle parameter is set to False, while bucket iterators by definition change the order of your data.
loss: 1.0171 ||: 100%|██████████| 1/1 [00:00<00:00, 125.22it/s]
loss: 1.0167 ||: 100%|██████████| 1/1 [00:00<00:00, 125.51it/s]
  0%|          | 0/1 [00:00<?, ?it/s]shuffle parameter is set to False, while bucket iterators by definition change the order of your data.
loss: 1.0164 ||: 100%|██████████| 1/1 [00:00<00:00, 233.15it/s]
loss: 1.0160 ||: 100%|██████████| 1/1 [00:00<00:00, 111.63it/s]
  0%|          | 0/1 [00:00<?, ?it/s]shuffle parameter is set to False, while bucket iterators by definition change the order of your data.
loss: 1.0157 ||: 100%|██████████| 1/1 [00:00<00:00, 155.

...

loss: 0.0237 ||: 100%|██████████| 1/1 [00:00<00:00, 174.92it/s]
loss: 0.0238 ||: 100%|██████████| 1/1 [00:00<00:00, 125.99it/s]
  0%|          | 0/1 [00:00<?, ?it/s]shuffle parameter is set to False, while bucket iterators by definition change the order of your data.
loss: 0.0236 ||: 100%|██████████| 1/1 [00:00<00:00, 177.63it/s]
loss: 0.0237 ||: 100%|██████████| 1/1 [00:00<00:00, 117.52it/s]
  0%|          | 0/1 [00:00<?, ?it/s]shuffle parameter is set to False, while bucket iterators by definition change the order of your data.
loss: 0.0236 ||: 100%|██████████| 1/1 [00:00<00:00, 204.00it/s]
loss: 0.0236 ||: 100%|██████████| 1/1 [00:00<00:00, 114.18it/s]
  0%|          | 0/1 [00:00<?, ?it/s]shuffle parameter is set to False, while bucket iterators by definition change the order of your data.
loss: 0.0235 ||: 100%|██████████| 1/1 [00:00<00:00, 216.51it/s]
loss: 0.0236 ||: 100%|██████████| 1/1 [00:00<00:00, 127.82it/s]
  0%|          | 0/1 [00:00<?, ?it/s]shuffle parameter is set to Fal

  0%|          | 0/1 [00:00<?, ?it/s]shuffle parameter is set to False, while bucket iterators by definition change the order of your data.
loss: 0.0219 ||: 100%|██████████| 1/1 [00:00<00:00, 198.06it/s]
loss: 0.0220 ||: 100%|██████████| 1/1 [00:00<00:00, 122.55it/s]
  0%|          | 0/1 [00:00<?, ?it/s]shuffle parameter is set to False, while bucket iterators by definition change the order of your data.
loss: 0.0219 ||: 100%|██████████| 1/1 [00:00<00:00, 223.04it/s]
loss: 0.0219 ||: 100%|██████████| 1/1 [00:00<00:00, 112.63it/s]
  0%|          | 0/1 [00:00<?, ?it/s]shuffle parameter is set to False, while bucket iterators by definition change the order of your data.
loss: 0.0218 ||: 100%|██████████| 1/1 [00:00<00:00, 210.39it/s]
loss: 0.0219 ||: 100%|██████████| 1/1 [00:00<00:00, 111.71it/s]
  0%|          | 0/1 [00:00<?, ?it/s]shuffle parameter is set to False, while bucket iterators by definition change the order of your data.
loss: 0.0218 ||: 100%|██████████| 1/1 [00:00<00:00, 176.

loss: 0.0204 ||: 100%|██████████| 1/1 [00:00<00:00, 234.58it/s]
loss: 0.0205 ||: 100%|██████████| 1/1 [00:00<00:00, 124.99it/s]
  0%|          | 0/1 [00:00<?, ?it/s]shuffle parameter is set to False, while bucket iterators by definition change the order of your data.
loss: 0.0204 ||: 100%|██████████| 1/1 [00:00<00:00, 163.89it/s]
loss: 0.0205 ||: 100%|██████████| 1/1 [00:00<00:00, 115.84it/s]
  0%|          | 0/1 [00:00<?, ?it/s]shuffle parameter is set to False, while bucket iterators by definition change the order of your data.
loss: 0.0204 ||: 100%|██████████| 1/1 [00:00<00:00, 197.06it/s]
loss: 0.0204 ||: 100%|██████████| 1/1 [00:00<00:00, 124.43it/s]
  0%|          | 0/1 [00:00<?, ?it/s]shuffle parameter is set to False, while bucket iterators by definition change the order of your data.
loss: 0.0203 ||: 100%|██████████| 1/1 [00:00<00:00, 160.04it/s]
loss: 0.0204 ||: 100%|██████████| 1/1 [00:00<00:00, 113.10it/s]
  0%|          | 0/1 [00:00<?, ?it/s]shuffle parameter is set to Fal

  0%|          | 0/1 [00:00<?, ?it/s]shuffle parameter is set to False, while bucket iterators by definition change the order of your data.
loss: 0.0191 ||: 100%|██████████| 1/1 [00:00<00:00, 218.24it/s]
loss: 0.0192 ||: 100%|██████████| 1/1 [00:00<00:00, 121.55it/s]
  0%|          | 0/1 [00:00<?, ?it/s]shuffle parameter is set to False, while bucket iterators by definition change the order of your data.
loss: 0.0191 ||: 100%|██████████| 1/1 [00:00<00:00, 186.38it/s]
loss: 0.0191 ||: 100%|██████████| 1/1 [00:00<00:00, 115.89it/s]
  0%|          | 0/1 [00:00<?, ?it/s]shuffle parameter is set to False, while bucket iterators by definition change the order of your data.
loss: 0.0190 ||: 100%|██████████| 1/1 [00:00<00:00, 198.24it/s]
loss: 0.0191 ||: 100%|██████████| 1/1 [00:00<00:00, 125.74it/s]
  0%|          | 0/1 [00:00<?, ?it/s]shuffle parameter is set to False, while bucket iterators by definition change the order of your data.
loss: 0.0190 ||: 100%|██████████| 1/1 [00:00<00:00, 147.

{'best_epoch': 999,
 'best_validation_loss': 0.01820364035665989,
 'training_duration': '00:00:59',
 'training_epochs': 1000,
 'training_loss': 0.018291914835572243,
 'training_start_epoch': 0,
 'validation_loss': 0.01820364035665989}

In [70]:
# NOTE : 
raw_train_generator = iterator(train_dataset,num_epochs = 10, shuffle=True)

for batch in raw_train_generator:
    print(batch)
    
# Dict[str,torch.Tensor]    

{'sentence': {'tokens': tensor([[ 7,  8,  9, 10,  0],
        [ 2,  3,  4,  5,  6]])}, 'labels': tensor([[0, 2, 1, 0, 0],
        [1, 0, 2, 1, 0]])}
{'sentence': {'tokens': tensor([[ 7,  8,  9, 10,  0],
        [ 2,  3,  4,  5,  6]])}, 'labels': tensor([[0, 2, 1, 0, 0],
        [1, 0, 2, 1, 0]])}
{'sentence': {'tokens': tensor([[ 7,  8,  9, 10,  0],
        [ 2,  3,  4,  5,  6]])}, 'labels': tensor([[0, 2, 1, 0, 0],
        [1, 0, 2, 1, 0]])}
{'sentence': {'tokens': tensor([[ 7,  8,  9, 10,  0],
        [ 2,  3,  4,  5,  6]])}, 'labels': tensor([[0, 2, 1, 0, 0],
        [1, 0, 2, 1, 0]])}
{'sentence': {'tokens': tensor([[ 7,  8,  9, 10,  0],
        [ 2,  3,  4,  5,  6]])}, 'labels': tensor([[0, 2, 1, 0, 0],
        [1, 0, 2, 1, 0]])}
{'sentence': {'tokens': tensor([[ 7,  8,  9, 10,  0],
        [ 2,  3,  4,  5,  6]])}, 'labels': tensor([[0, 2, 1, 0, 0],
        [1, 0, 2, 1, 0]])}
{'sentence': {'tokens': tensor([[ 7,  8,  9, 10,  0],
        [ 2,  3,  4,  5,  6]])}, 'labels': tensor([[

## Step 5.  Prediction (Verification)

In [71]:
from allennlp.predictors import SentenceTaggerPredictor


In [75]:
predictor = SentenceTaggerPredictor(model, dataset_reader = reader)

tag_logits = predictor.predict("The dog ate the apple")['tag_logits']

tag_ids = np.argmax(tag_logits, axis=-1)

print ("The dog ate the apple")
print([model.vocab.get_token_from_index(i,'labels') for i in tag_ids])

The dog ate the apple
['DET', 'NN', 'V', 'DET', 'NN']
